In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

In [2]:
options = Options() 
chrome_path = '/Users/User/Documents/chromedriver'

In [3]:
options.add_argument('--window-size=500,300')

In [4]:
driver = webdriver.Chrome(chrome_path, 
                              options=options)

driver.set_window_size(1400,1000)

In [15]:
driver.get('http://101.78.205.36/racing/Info/meeting/RaceCard/english/Local/20200223/ST/1')
time.sleep(7)

In [16]:
page_source = driver.page_source
soup = BeautifulSoup(page_source)
tables = soup.findAll('table', attrs = {'class' : 'draggable hiddenable'})
table = tables[0].prettify()
df = pd.read_html(table)[0]

In [17]:
page_source = driver.page_source
soup = BeautifulSoup(page_source)
tables = soup.findAll('table', attrs = {'class' : 'font13 lineH20 tdAlignL'})
table = tables[0].prettify()
race_info = pd.read_html(table)[0]

In [18]:
def month_string_to_number(string):
    m = {
        'jan': '01',
        'feb': '02',
        'mar': '03',
        'apr': '04',
         'may': '05',
         'jun': '06',
         'jul': '07',
         'aug': '08',
         'sep': '09',
         'oct': '10',
         'nov': '11',
         'dec': '12'
        }
    s = string.strip()[:3].lower()

    try:
        out = m[s]
        return out
    except:
        raise ValueError('Not a month')

In [19]:
def add_race_info(ri):
    rinfo = race_info[0].values[0]
    try:
        race_class = rinfo.split('Class ')[1]
    except:
        try:
            race_class = rinfo.split('Group ')[1]
        except:
            race_class = rinfo.split('4 Year')
            race_class = '4YO'
    if 'Turf' in race_info[0].values[0]:
        rinfo.split('Turf')
        df['Race  Class'] = race_class
        dist = rinfo.split(', ')[6]
        dist = dist.split()
        dist = dist[0].replace('M','')
        df['Dist.'] = dist
        course = rinfo.split(', ')[5]
        course = course.split()
        course = course[0].replace('"', '')
        df['Course'] = course
        track = rinfo.split(', ')[4]
        track = track.split()
        track = track[1]
        df['Track'] = track
        rc = rinfo.split(', ')[3]
        rc = rc.split()
        rc = rc[0][0] + rc[1][0]
        df['RC'] = rc
        date = rinfo.split(', ')[1]
        date = date + ' ' + rinfo.split(', ')[2]
        date = date.split()
        month = month_string_to_number(date[0])
        day = date[1]
        year = date[2]
        year = year[:2]
        date = day + '/' + month + '/' + year
        df['Date'] = date
    else:
        df['Race  Class'] = race_class
        dist = rinfo.split(', ')[5]
        dist = dist.split()
        dist = dist[0].replace('M','')
        df['Dist.'] = dist
        df['Course'] = np.nan
        df['Track'] = 'AWT'
        rc = rinfo.split(', ')[3]
        rc = rc.split()
        rc = rc[0][0] + rc[1][0]
        df['RC'] = rc
        date = rinfo.split(', ')[1]
        date = date + ' ' + rinfo.split(', ')[2]
        date = date.split()
        month = month_string_to_number(date[0])
        day = date[1]
        year = date[2]
        year = year[:2]
        date = day + '/' + month + '/' + year
        df['Date'] = date

In [20]:
add_race_info(race_info)

In [21]:
df.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13],inplace = True)

In [25]:
g = df

In [23]:
races = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10','11']
raceno = 1
for race in races:
    try:
        time.sleep(2)
        driver.get('http://101.78.205.36/racing/Info/meeting/RaceCard/english/Local/')
        time.sleep(5)
        if int(race) > 1:
            driver.find_element_by_id('racecard_rt_' + race + '_r').click()
            time.sleep(5)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source)
            tables = soup.findAll('table', attrs = {'class' : 'draggable hiddenable'})
            table = tables[0].prettify()
            df = pd.read_html(table)[0]
            time.sleep(2)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source)
            tables = soup.findAll('table', attrs = {'class' : 'font13 lineH20 tdAlignL'})
            table = tables[0].prettify()
            race_info = pd.read_html(table)[0]
            add_race_info(race_info)
            df['Race Index'] = str(raceno)
            raceno = raceno + 1
            g = pd.concat([g,df], ignore_index = True, sort = True)
        else:
            time.sleep(5)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source)
            tables = soup.findAll('table', attrs = {'class' : 'draggable hiddenable'})
            table = tables[0].prettify()
            df = pd.read_html(table)[0]
            time.sleep(2)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source)
            tables = soup.findAll('table', attrs = {'class' : 'font13 lineH20 tdAlignL'})
            table = tables[0].prettify()
            race_info = pd.read_html(table)[0]
            add_race_info(race_info)
            df['Race Index'] = str(raceno)
            raceno = raceno + 1
            g = pd.concat([g,df], ignore_index = True, sort = True)
    except:
        continue

In [50]:
g.columns

Index(['Colour', 'Course', 'Date', 'Dist.', 'Draw', 'Gear', 'Horse',
       'Horse No.', 'Horse Wt. (Declaration)', 'Jockey', 'Last 6 Runs',
       'Priority', 'RC', 'Race  Class', 'Race Index', 'Rtg.', 'Rtg.+/-',
       'Track', 'Trainer', 'Wt.'],
      dtype='object')

In [51]:
g.drop('Colour', axis = 1, inplace = True)
g.drop('Horse No.', axis = 1, inplace = True)
g.drop('Last 6 Runs', axis = 1, inplace = True)
g.drop('Priority', axis = 1, inplace = True)
g.drop('Rtg.+/-', axis = 1, inplace = True)
g.drop('Rtg.', axis = 1, inplace = True)

In [52]:
g.drop('Gear', axis = 1, inplace = True)

In [55]:
g.rename(columns = {'Horse' : 'hname', 'Wt.' : 'Act.  Wt.',
                   'Horse Wt. (Declaration)' : 'Declar.  Horse Wt.',
                   'Draw':'Dr'}, inplace = True)

In [56]:
hinfo = pd.read_csv('m_horse_info.csv')

In [57]:
future_races = g.join(hinfo.set_index('hname'), on='hname')

In [58]:
future_races.to_csv('future_races.csv', index = False)

In [59]:
driver.quit()